In [3]:
from dotenv import load_dotenv
load_dotenv()

import pandas as pd
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from langchain_google_genai import ChatGoogleGenerativeAI

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
#"gemini-2.0-flash-thinking-exp-01-21"
#config = {'thinking_config': {'include_thoughts': True}}

#"gemini-2.0-flash-exp"

llm = ChatGoogleGenerativeAI(model="gemini-2.0-pro-exp-02-05", temperature=0)
df = pd.read_excel("query_result_2025-01-25T07_49_01.047868Z.xlsx")
df = df[df.abstract.notna()]

/usr/local/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [5]:
with open("prompt.txt", "r") as f:
    prompt = f.read()
prompt = PromptTemplate(input_variables=['PROPOSAL_INFO'],template=prompt)

chain = prompt | llm | JsonOutputParser()

proposal_info_columns = [ 'title'
  , 'abstract'
  , 'detailed_description'
  , 'outline'
  , 'objective']

In [6]:
result = []

In [7]:
#4 RPM
import time

processed_proposals = set(r.get('proposal_id') for r in result)
print(processed_proposals)

for proposal_id in df.proposal_id.unique():
    # Skip if already processed
    if proposal_id in processed_proposals:
        print(f"Skipping already processed proposal: {proposal_id}")
        continue
        
    print(f"Processing proposal: {proposal_id}")
    proposal_info = df[df.proposal_id == proposal_id][proposal_info_columns][:1].to_dict(orient='records')[0]
    proposal_info_str = [f'{x}: {proposal_info[x]}' for x in proposal_info_columns]
    chain = prompt | llm | JsonOutputParser()
    executive_method = chain.invoke({"PROPOSAL_INFO": proposal_info_str})
    
    # Add proposal_id to the result for tracking
    executive_method['proposal_id'] = proposal_id
    result.append(executive_method)
    time.sleep(20)

set()
Processing proposal: 2185846315039064832
Processing proposal: 2185844604283126528
Processing proposal: 2205136900467983104
Processing proposal: 2208409881428361984
Processing proposal: 2185842091785978624
Processing proposal: 2185845802830660352
Processing proposal: 2197952451796009728
Processing proposal: 2181661100230050560
Processing proposal: 2209040357629362944
Processing proposal: 2188525315913941760
Processing proposal: 2197262136667800320
Processing proposal: 2209120431347073792
Processing proposal: 2201815482904871680
Processing proposal: 2203075043099935488
Processing proposal: 2183455404749488896
Processing proposal: 2180814504890204928
Processing proposal: 2202583607111844608


In [9]:
from collections import Counter

Counter([ x['vote'] for x in result])


Counter({'+0': 11, '-0': 4, '+1': 1, '-1': 1})

# flash thinking

In [160]:
import os
from google import genai

client = genai.Client(api_key=os.environ["GOOGLE_API_KEY"], http_options={'api_version':'v1alpha'})

config = {'thinking_config': {'include_thoughts': True}, 'temperature': 0}


In [162]:
thinking_result = []

for proposal_id in df.proposal_id.unique():
    print(proposal_id)
    proposal_info = df[df.proposal_id == proposal_id][proposal_info_columns][:1].to_dict(orient='records')[0]
    proposal_info_str = [f'{x}: {proposal_info[x]}' for x in proposal_info_columns]

    response = client.models.generate_content(
        model='gemini-2.0-flash-thinking-exp',
        contents=prompt.invoke(
            {"PROPOSAL_INFO": proposal_info_str}).text,
        config=config
)

    thinking_result.append(response)

2185846315039064832
2185844604283126528
2205136900467983104
2208409881428361984
2185842091785978624
2185845802830660352
2197952451796009728
2181661100230050560
2209040357629362944
2188525315913941760
2197262136667800320
2209120431347073792
2201815482904871680
2203075043099935488
2183455404749488896
2180814504890204928
2202583607111844608


In [169]:
print(thinking_result[0].text)

```json
{
  "comment": "提案主題關於 LLM 評估很有趣，但與 Python 的關聯性較弱。建議明確說明如何使用 Python 進行 LLM 實驗、數據分析或模型互動。例如，可以展示 Python 程式碼範例，介紹相關的 Python 函式庫，並深入探討 Python 在 LLM 評估中的技術細節。加強 Python 技術深度將使提案更符合 PyCon TW 主題。",
  "vote": "-0"
}
```


In [171]:
import json

parsed_result = [json.loads(x.text.strip('```json').strip('```').strip()) for x in thinking_result]

In [176]:
from collections import Counter
Counter([ x['vote'] for x in parsed_result])



Counter({'+0': 9, '-0': 4, '+1': 3, '-1': 1})

In [ ]:
Counter({'+0': 10, '-0': 4, '+1': 2, '-1': 1})
Counter({'+0': 9, '-0': 4, '+1': 3, '-1': 1}) #thinking

# Get result

In [10]:
pivot_df = df.pivot_table(
    index='proposal_id',
    columns=df.groupby('proposal_id').cumcount(),
    values='comment',
    aggfunc='first'
)
pivot_df.columns = [f'comment_{i+1}' for i in range(pivot_df.shape[1])]
pivot_df.reset_index(inplace=True)


In [11]:
# Create DataFrame from LLM results
regular_result_df = pd.DataFrame(result)

# Rename columns to distinguish results
regular_result_df = regular_result_df.add_prefix('LLM_')

proposal_df = df[['proposal_id'] + proposal_info_columns].drop_duplicates(subset=['proposal_id']).sort_values(by='proposal_id', ascending=False)

# pivot_df.merge(regular_result_df, left_on='proposal_id', right_on='LLM_proposal_id', how='left').merge(
#     proposal_df, on='proposal_id', how='left')

final_df = proposal_df.merge(regular_result_df, left_on='proposal_id', right_on='LLM_proposal_id', how='left').merge(
    pivot_df, on='proposal_id', how='left').sort_values(by='proposal_id', ascending=False)


In [12]:
final_df['human_eval'] = ''
final_df.to_excel('final_df_0208_pre.xlsx', index=False)

In [16]:
final_df[:3]

,proposal_id,title,abstract,detailed_description,outline,objective,LLM_comment,LLM_vote,LLM_proposal_id,comment_1,comment_2,comment_3,comment_4,comment_5,comment_6,comment_7,human_eval
0,2209120431347073792,Cracking the Code: Decoding Anti-Bot Systems!,Join me for a masterclass where I share the my...,Join me for a masterclass where I share the my...,"The presentation spans 35 minutes, followed by...","Web scraping is gaining momentum, particularly...","提案主題 ""Cracking the Code: Decoding Anti-Bot Sys...",+0,2209120431347073792,Thanks for this comprehensive proposal. I also...,I did look into the video talk. And together w...,NaN,NaN,NaN,NaN,NaN,
1,2209040357629362944,自動化電話錄音評測系統：提升超過 30 倍生產力,公司每天有上百通電話錄音需要進行稽核，以往僅能透過抽查的方式進行人工稽核，於是利用 STT(...,STT 工具：\r\nhttps://github.com/openai/whisper\r...,"5min 既有流程簡介\r\n5min 簡介 STT, LLM\r\n10min 程式碼分享...",幫助對於在客服或是電銷團隊能夠加速稽核流程，透過 Python 開源套件能在短時間建構出適合...,提案主題明確，展示了如何利用Python結合STT和LLM技術來自動化電話錄音評測，並顯著提...,+0,2209040357629362944,The outline is quite sparse and lacks substant...,這份提案介紹了一個自動化電話錄音評測系統，利用STT（Speech To Text）和LLM...,NaN,NaN,NaN,NaN,NaN,
2,2208409881428361984,Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢及報告生成平台,隨著企業越來越重視環境、社會和治理（ESG）的可持續發展目標，對於準確且全面的 ESG 資訊...,### Domain: ESG (環境、社會和治理)\r\nESG 指的是評估企業在環境保護...,* 引言：ESG的重要性、了解LLM與RAG [5 mins]\r\n * ESG對現...,本次演講目標是向以下受眾展示 LLM 和 RAG 技術在 ESG 資訊查詢和報告生成中的創新...,提案主題「Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢...,+0,2208409881428361984,看起來是在介紹LLM應用，但對於python主題看不出實際有何特殊處,The talk is undoubtedly fascinating and insigh...,NaN,NaN,NaN,NaN,NaN,


# append some info

In [36]:
vote_df = pd.read_excel('pycon_2025_raw_vote.xlsx', dtype={'vote': str})
vote_df['vote_int'] = vote_df['vote'].astype(int)

/usr/local/lib/python3.11/site-packages/openpyxl/styles/stylesheet.py:237: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [37]:
# Get statistics using vote_int (numeric) and vote (string) columns
vote_stats = vote_df.groupby('proposal_id', as_index=False).agg({
    'vote': [
        ('most_common_vote', lambda x: x.mode().iloc[0]),  # Most common vote as string
        ('vote_counts', lambda x: x.value_counts().to_dict())  # Distribution of votes
    ],
    'vote_int': [
        ('mean', 'mean'),     # Average vote
        ('std', 'std'),       # Standard deviation
        ('count', 'count'),   # Number of votes
        ('median', 'median')  # Median vote
    ]
})

# Flatten column names
vote_stats.columns = ['proposal_id' if col[0] == 'proposal_id' 
                     else f'{col[0]}_{col[1]}' for col in vote_stats.columns]

In [38]:
vote_stats

,proposal_id,vote_most_common_vote,vote_vote_counts,vote_int_mean,vote_int_std,vote_int_count,vote_int_median
0,2178936233474917120,+0,"{'+0': 7, '+1': 1}",0.125000,0.353553,8,0.0
1,2178979854999880448,+0,"{'+0': 4, '-0': 3, '+1': 2}",0.222222,0.440959,9,0.0
2,2178981707884004096,-0,"{'-0': 7, '+0': 4, '+1': 1}",0.083333,0.288675,12,0.0
3,2179613228097602304,+0,"{'+0': 6, '+1': 2}",0.250000,0.462910,8,0.0
4,2180814504890204928,-0,"{'-0': 5, '+0': 2, '-1': 1}",-0.125000,0.353553,8,0.0
...,...,...,...,...,...,...,...
79,2210517591254893312,+0,"{'+0': 6, '+1': 6, '-0': 2}",0.428571,0.513553,14,0.0
80,2210697815674323712,+1,"{'+1': 5, '+0': 4, '-0': 3}",0.416667,0.514929,12,0.0
81,2210750583986455296,+0,"{'+0': 7, '+1': 1}",0.125000,0.353553,8,0.0
82,2210778103544808192,+0,"{'+0': 5, '+1': 2, '-0': 2}",0.222222,0.440959,9,0.0


In [44]:
col_sort = ['proposal_id', 'title', 'abstract', 'detailed_description', 'outline',
       'objective', 'comment_1', 'comment_2', 'comment_3', 'comment_4', 'comment_5', 'comment_6', 'comment_7',
       'LLM_comment', 'LLM_vote',
       'vote_most_common_vote', 'vote_vote_counts',
       'vote_int_mean', 'vote_int_std', 'vote_int_count', 'vote_int_median',
       'human_eval']

final_df.merge(vote_stats, on='proposal_id', how='left')[col_sort][:3]

#vote_stats

,proposal_id,title,abstract,detailed_description,outline,objective,comment_1,comment_2,comment_3,comment_4,...,comment_7,LLM_comment,LLM_vote,vote_most_common_vote,vote_vote_counts,vote_int_mean,vote_int_std,vote_int_count,vote_int_median,human_eval
0,2209120431347073792,Cracking the Code: Decoding Anti-Bot Systems!,Join me for a masterclass where I share the my...,Join me for a masterclass where I share the my...,"The presentation spans 35 minutes, followed by...","Web scraping is gaining momentum, particularly...",Thanks for this comprehensive proposal. I also...,I did look into the video talk. And together w...,NaN,NaN,...,NaN,"提案主題 ""Cracking the Code: Decoding Anti-Bot Sys...",+0,+1,"{'+1': 5, '-0': 4, '+0': 3, '-1': 2}",0.214286,0.699293,14,0.0,
1,2209040357629362944,自動化電話錄音評測系統：提升超過 30 倍生產力,公司每天有上百通電話錄音需要進行稽核，以往僅能透過抽查的方式進行人工稽核，於是利用 STT(...,STT 工具：\r\nhttps://github.com/openai/whisper\r...,"5min 既有流程簡介\r\n5min 簡介 STT, LLM\r\n10min 程式碼分享...",幫助對於在客服或是電銷團隊能夠加速稽核流程，透過 Python 開源套件能在短時間建構出適合...,The outline is quite sparse and lacks substant...,這份提案介紹了一個自動化電話錄音評測系統，利用STT（Speech To Text）和LLM...,NaN,NaN,...,NaN,提案主題明確，展示了如何利用Python結合STT和LLM技術來自動化電話錄音評測，並顯著提...,+0,+0,"{'+0': 5, '-0': 4, '-1': 2}",-0.181818,0.404520,11,0.0,
2,2208409881428361984,Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢及報告生成平台,隨著企業越來越重視環境、社會和治理（ESG）的可持續發展目標，對於準確且全面的 ESG 資訊...,### Domain: ESG (環境、社會和治理)\r\nESG 指的是評估企業在環境保護...,* 引言：ESG的重要性、了解LLM與RAG [5 mins]\r\n * ESG對現...,本次演講目標是向以下受眾展示 LLM 和 RAG 技術在 ESG 資訊查詢和報告生成中的創新...,看起來是在介紹LLM應用，但對於python主題看不出實際有何特殊處,The talk is undoubtedly fascinating and insigh...,NaN,NaN,...,NaN,提案主題「Eco Smart Guide: 透過 Python 打造一站式 ESG 資訊查詢...,+0,+0,"{'+0': 7, '-1': 2, '+1': 1, '-0': 1}",-0.090909,0.539360,11,0.0,


In [56]:
# Get the merged dataframe with all necessary columns
merged_df = final_df.merge(vote_stats, on='proposal_id', how='left')[col_sort]


# Calculate agreement between LLM and human most common vote
agreement_count = (merged_df['LLM_vote'] == merged_df['vote_most_common_vote']).sum()
total_proposals = len(merged_df)
agreement_rate = agreement_count / total_proposals

print(f"AI 與人類的主流意見一致率: {agreement_rate:.2f}")

# Optional: Detailed analysis
print("\n詳細分析:")
print("AI 評分分布:")
print(merged_df['LLM_vote'].value_counts())
print("\n人類中位數評分分布:")
print(merged_df['vote_most_common_vote'].value_counts())

AI 與人類的主流意見一致率: 0.29

詳細分析:
AI 評分分布:
LLM_vote
+0    11
-0     4
+1     1
-1     1
Name: count, dtype: int64

人類中位數評分分布:
vote_most_common_vote
-0    6
+0    5
+1    3
-1    3
Name: count, dtype: int64


In [57]:
merged_df.shape

(17, 22)

In [58]:
# 先檢查 DataFrame 的結構
print("=== DataFrame Columns ===")
print(merged_df.columns.tolist())

# 修改後的檢查函數
def check_vote_distribution(merged_df):
    print("\n=== Vote Distribution Analysis ===")
    
    # LLM votes distribution
    print("\nLLM Vote Distribution:")
    print(merged_df['LLM_vote'].value_counts(normalize=True).round(3))
    
    # Human votes distribution
    print("\nHuman Vote Distribution:")
    print(merged_df['vote_most_common_vote'].value_counts(normalize=True).round(3))
    
    # Vote agreement analysis
    agreement = (merged_df['LLM_vote'] == merged_df['vote_most_common_vote'])
    print(f"\nOverall Agreement Rate: {agreement.mean():.3f}")
    
    # Disagreement cases analysis
    disagreement_df = merged_df
    if len(disagreement_df) > 0:
        print("\nAgreement Cases Analysis:")
        print(pd.crosstab(
            disagreement_df['LLM_vote'], 
            disagreement_df['vote_most_common_vote'],
            margins=True
        ))

# temp not using this part

# def check_vote_variance(merged_df):
#     print("\n=== Vote Variance Analysis ===")
    
#     # 假設標準差欄位名稱是 ('vote_int', 'std')
#     std_column = ('vote_int', 'std')
#     if std_column in merged_df.columns:
#         high_std_threshold = merged_df[std_column].quantile(0.75)
        
#         high_variance_cases = merged_df[merged_df[std_column] > high_std_threshold]
#         print(f"\nHigh Variance Cases (std > {high_std_threshold:.3f}):")
#         print(f"Number of cases: {len(high_variance_cases)}")
        
#         if len(high_variance_cases) > 0:
#             print("\nSample of high variance cases:")
#             # 調整要顯示的欄位
#             relevant_columns = ['proposal_id', std_column, 'LLM_vote', 'vote_most_common_vote']
#             print(high_variance_cases[relevant_columns].head())
#     else:
#         print("Note: Standard deviation column not found in the DataFrame")
#         print("Available columns:", merged_df.columns.tolist())

# 執行檢查
check_vote_distribution(merged_df)

=== DataFrame Columns ===
['proposal_id', 'title', 'abstract', 'detailed_description', 'outline', 'objective', 'comment_1', 'comment_2', 'comment_3', 'comment_4', 'comment_5', 'comment_6', 'comment_7', 'LLM_comment', 'LLM_vote', 'vote_most_common_vote', 'vote_vote_counts', 'vote_int_mean', 'vote_int_std', 'vote_int_count', 'vote_int_median', 'human_eval']

=== Vote Distribution Analysis ===

LLM Vote Distribution:
LLM_vote
+0    0.647
-0    0.235
+1    0.059
-1    0.059
Name: proportion, dtype: float64

Human Vote Distribution:
vote_most_common_vote
-0    0.353
+0    0.294
+1    0.176
-1    0.176
Name: proportion, dtype: float64

Overall Agreement Rate: 0.294

Agreement Cases Analysis:
vote_most_common_vote  +0  +1  -0  -1  All
LLM_vote                                  
+0                      4   3   4   0   11
+1                      1   0   0   0    1
-0                      0   0   1   3    4
-1                      0   0   1   0    1
All                     5   3   6   3   17


In [60]:
merged_df.to_excel('merged_df_0208.xlsx')